In [5]:
import pandas as pd
from utils import *

In [6]:
# -- SALES_TARGET_DATA doeltabel --

# SALES_TARGET_DATA_ID
# SALES_TARGET_DATA_SALES-TARGET
# SALES_TARGET_DATA_PRODUCTIVITY_SCORE* (calculateSPS(): (UNIT_PRICE * QUANTITY) / SALES_TARGET)

# ORDER_DETAILS_UNIT_COST
# ORDER_DETAILS_UNIT_PRICE
# ORDER_DETAILS_UNIT_SALE_PRICE
# ORDER_DETAILS_QUANTITY

# SALES_STAFF_CODE
# SALES_BRANCH_CODE
# PRODUCT_NUMBER
# RETAILER_CODE

In [7]:
go_sales = load_db('source/go_sales.sqlite')
go_crm = load_db('source/go_crm.sqlite')

In [8]:
sales_target_data = load_table(go_sales, 'SALES_TARGETData')
order_details = load_table(go_sales, 'ORDER_DETAILS')
sales_staff = load_table(go_sales, 'SALES_STAFF')
sales_branch = load_table(go_sales, 'SALES_BRANCH')
product = load_table(go_sales, 'PRODUCT')
retailer = load_table(go_crm, 'RETAILER')